In [2]:
# H7_feedforward.py
import numpy as np
import pandas as pd
import tensorflow as tf
import time

In [3]:
# load training and testing notMNIST data
MNIST = np.load('./MNIST_train_40000.npz')
images_train = MNIST['train_images']
labels_train = MNIST['train_labels']
MNIST = np.load('./MNIST_test_10000.npz')
images_test  = MNIST['test_images']
labels_test  = MNIST['test_labels']
print('training')
print(images_train.shape)
print(labels_train.shape)
print()
print('testing')
print(images_test.shape)
print(labels_test.shape)

training
(40000, 28, 28)
(40000,)

testing
(10000, 28, 28)
(10000,)


In [4]:
# define inputs and outputs
inputs_train  = images_train.reshape(40000,28*28)
inputs_test  = images_test.reshape(10000,28*28)
df = pd.DataFrame(labels_train,columns=['digits'],dtype='category')
outputs_train = pd.get_dummies(df.digits)
df = pd.DataFrame(labels_test,columns=['digits'],dtype='category')
outputs_test = pd.get_dummies(df.digits)

In [5]:
# define TensorFlow computation graph
X  = tf.placeholder(dtype='float32',shape=[40000,28*28])
Xt = tf.placeholder(dtype='float32',shape=[10000,28*28])
P  = tf.placeholder(dtype='float32',shape=[40000,10])
Pt = tf.placeholder(dtype='float32',shape=[10000,10])
W1 = tf.Variable(tf.truncated_normal([28*28,128], stddev=0.1))
b1 = tf.Variable(tf.truncated_normal([1,128], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([128,128], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([1,128], stddev=0.1))
W3 = tf.Variable(tf.truncated_normal([128,10], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([1,10], stddev=0.1))
Y1  = tf.nn.relu(tf.matmul(X,W1) + b1)
Y1t = tf.nn.relu(tf.matmul(Xt,W1) + b1)
Y2  = tf.nn.relu(tf.matmul(Y1,W2) + b2)
Y2t = tf.nn.relu(tf.matmul(Y1t,W2) + b2)
Y3  = tf.matmul(Y1,W3) + b3
Y3t = tf.matmul(Y1t,W3) + b3
CE  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y3,labels=P))
CEt = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y3t,labels=Pt))
optimizer = tf.train.AdamOptimizer().minimize(CE)
Po  = tf.nn.softmax(Y3)
Pot = tf.nn.softmax(Y3t)
init = tf.global_variables_initializer()
sess = tf.Session()

In [6]:
# run TensorFlow computation graph
sess.run(init)
MAXSTEPS = 1000
MOD = MAXSTEPS/10
time_start = time.time()
for step in range(MAXSTEPS+1):
    (_,ce,cet) = sess.run([optimizer,CE,CEt],
                          feed_dict={X:inputs_train,Xt:inputs_test,P:outputs_train,Pt:outputs_test})
    if (step % MOD) == 0:
        time_elapsed   = time.time() - time_start
        time_per_iter  = time_elapsed/(step + 1)
        time_remaining = (MAXSTEPS - step)*time_per_iter
        print('step %6d of %6d  train CE %-.5f  test CE %-.5f  time remaining (min) %-.1f' % 
              (step,MAXSTEPS,ce,cet,time_remaining/60))


step      0 of   1000  train CE 2.40733  test CE 2.41172  time remaining (min) 32.9
step    100 of   1000  train CE 0.25437  test CE 0.27842  time remaining (min) 29.8


KeyboardInterrupt: 

In [ ]:
# compute error rates
(po,pot) = sess.run([Po,Pot],feed_dict={X:inputs_train,Xt:inputs_test})
p  = outputs_train.values
pt = outputs_test.values
err_train =  (po.argmax(axis=1)  != p.argmax(axis=1)).sum().sum()
err_test  =  (pot.argmax(axis=1) != pt.argmax(axis=1)).sum().sum()
err_train = err_train/p.shape[0]
err_test  = err_test/pt.shape[0]
print()
print('training error rate = ',err_train.round(3),'  testing error rate = ',err_test.round(3))
print()
time_total = time.time() - time_start
print('total training time (min)',np.round(time_total/60,1))